# Assignment 4
# Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from assets/wikipedia_data.html and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in assets/nfl.csv), MLB (baseball, in assets/mlb.csv), NBA (basketball, in assets/nba.csv or NHL (hockey, in assets/nhl.csv). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: what is the win/loss ratio's correlation with the population of the city it is in? Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with pearsonr, so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%*4=80%) of the grade for this assignment. You should only use data from year 2018 for your analysis -- this is important!

# Notes
Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

# Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NHL using 2018 data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [4]:
nhl_df=pd.read_csv(r"C:\Users\Юрий\Desktop\Занятия\Coursera\assignments\assignment4\assets\nhl.csv")
cities=pd.read_html(r"C:\Users\Юрий\Desktop\Занятия\Coursera\assignments\assignment4\assets\wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [3]:
cities.replace('\[.+','',regex=True, inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"([A-Z])", r" \1").str.strip()
cities['MLB'] = cities['MLB'].str.replace(r"([A-Z])", r" \1").str.strip()
cities['NBA'] = cities['NBA'].str.replace(r"([A-Z])", r" \1").str.strip()
cities['NHL'] = cities['NHL'].str.replace(r"([A-Z])", r" \1").str.strip()
cities.rename(columns={"Population (2016 est.)[8]": "Population"},
              inplace=True)
cities['NHL1'] = cities['NHL'].str.split().str[-1]
cities['NHL2'] = cities['NHL'].str.split().str[-2]
cities['NHL3'] = cities['NHL'].str.split().str[-3]
cities

,Metropolitan area,Population,NFL,MLB,NBA,NHL,NHL1,NHL2,NHL3
0,New York City,20153634,Giants Jets,Yankees Mets,Knicks Nets,Rangers Islanders Devils,Devils,Islanders,Rangers
1,Los Angeles,13310447,Rams Chargers,Dodgers Angels,Lakers Clippers,Kings Ducks,Ducks,Kings,NaN
2,San Francisco Bay Area,6657982,49ers Raiders,Giants Athletics,Warriors,Sharks,Sharks,NaN,NaN
3,Chicago,9512999,Bears,Cubs White Sox,Bulls,Blackhawks,Blackhawks,NaN,NaN
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars,Stars,NaN,NaN
5,"Washington, D.C.",6131977,Redskins,Nationals,Wizards,Capitals,Capitals,NaN,NaN
6,Philadelphia,6070500,Eagles,Phillies,76ers,Flyers,Flyers,NaN,NaN
7,Boston,4794447,Patriots,Red Sox,Celtics,Bruins,Bruins,NaN,NaN
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild,Wild,NaN,NaN
9,Denver,2853077,Broncos,Rockies,Nuggets,Avalanche,Avalanche,NaN,NaN


In [4]:
nhl_df['team'].replace('\*','',regex=True, inplace=True) # убрали все сноски в названиях команд (напр.*[note2])
nhl_df =nhl_df[nhl_df['year']==2018] # выбрали только 2018й год.  
nhl_df = nhl_df[nhl_df['GP']=='82']  # убрали разделение на территориальные лиги (объединили лиги в один список) 

In [5]:
cols = ['GP', 'W', 'L', 'OL', 'PTS', 'PTS%', 'GF', 'GA', 'SRS', 'SOS',
       'RPt%', 'ROW', 'year']
nhl_df[cols] = nhl_df[cols].apply(pd.to_numeric, errors='coerce') # перевели все числовые значения в int
nhl_df['W/L%'] = nhl_df['W']/(nhl_df['W']+nhl_df['L'])
nhl_df

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,W/L%
1,Tampa Bay Lightning,82,54,23,5,113,0.689,296,236,0.66,-0.07,0.634,48,2018,NHL,0.701299
2,Boston Bruins,82,50,20,12,112,0.683,270,214,0.62,-0.07,0.610,47,2018,NHL,0.714286
3,Toronto Maple Leafs,82,49,26,7,105,0.640,277,232,0.49,-0.06,0.567,42,2018,NHL,0.653333
4,Florida Panthers,82,44,30,8,96,0.585,248,246,-0.01,-0.04,0.537,41,2018,NHL,0.594595
5,Detroit Red Wings,82,30,39,13,73,0.445,217,255,-0.48,-0.01,0.341,25,2018,NHL,0.434783
6,Montreal Canadiens,82,29,40,13,71,0.433,209,264,-0.68,0.00,0.378,27,2018,NHL,0.420290
7,Ottawa Senators,82,28,43,11,67,0.409,221,291,-0.85,0.00,0.372,26,2018,NHL,0.394366
8,Buffalo Sabres,82,25,45,12,62,0.378,199,280,-0.98,0.01,0.311,24,2018,NHL,0.357143
10,Washington Capitals,82,49,26,7,105,0.640,259,239,0.21,-0.04,0.585,46,2018,NHL,0.653333
11,Pittsburgh Penguins,82,47,29,6,100,0.610,272,250,0.23,-0.04,0.573,45,2018,NHL,0.618421


In [6]:
team_pop = nhl_df['team'].str.extract('(\w+$)')
team_pop.rename(columns = {0 : 'Name team'}, inplace = True)
matches = cities['NHL'].str.split().str[-1]
matches2 = cities['NHL'].str.split().str[-2]
matches3 = cities['NHL'].str.split().str[-3]
matches = team_pop['Name team'].isin(nhl_df['team'].str.split().str[-1])
team_pop = pd.merge(team_pop, nhl_df[['team','W/L%']], how='inner',left_on='Name team', right_on=nhl_df['team'].str.split().str[-1])
team_pop = pd.merge(team_pop,cities[['Metropolitan area','Population']], how='left',left_on='Name team',right_on=cities['NHL1'])
team_pop = pd.merge(team_pop,cities[['Metropolitan area','Population']], how='left',left_on='Name team',right_on=cities['NHL2'])
team_pop = pd.merge(team_pop,cities[['Metropolitan area','Population']], how='left',left_on='Name team',right_on=cities['NHL3'])
team_pop= team_pop.fillna('')
team_pop['Metropolitan area'] = team_pop['Metropolitan area'] +team_pop['Metropolitan area_x'] +team_pop['Metropolitan area_y'] 
team_pop['Population'] = team_pop['Population'] +team_pop['Population_x'] +team_pop['Population_y']
team_pop.drop(['Name team', 'Metropolitan area_x','Metropolitan area_y','Population_x','Population_y'], axis=1, inplace=True) 
team_pop[['W/L%','Population']] = team_pop[['W/L%','Population']].apply(pd.to_numeric, errors='coerce')
res_df = team_pop.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
population_by_region = res_df['Population'] 
win_loss_by_region =res_df['W/L%']
#return 
stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.012486162921209895

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
def nhl_correlation():
    nhl_df=pd.read_csv(r"C:\Users\Юрий\Desktop\Занятия\Coursera\assignments\assignment4\assets\nhl.csv")
    cities=pd.read_html(r"C:\Users\Юрий\Desktop\Занятия\Coursera\assignments\assignment4\assets\wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.replace('\[.+','',regex=True, inplace=True)
    cities['NFL'] = cities['NFL'].str.replace(r"([A-Z])", r" \1").str.strip()
    cities['MLB'] = cities['MLB'].str.replace(r"([A-Z])", r" \1").str.strip()
    cities['NBA'] = cities['NBA'].str.replace(r"([A-Z])", r" \1").str.strip()
    cities['NHL'] = cities['NHL'].str.replace(r"([A-Z])", r" \1").str.strip()
    cities.rename(columns={"Population (2016 est.)[8]": "Population"},
                  inplace=True)
    cities['NHL1'] = cities['NHL'].str.split().str[-1]
    cities['NHL2'] = cities['NHL'].str.split().str[-2]
    cities['NHL3'] = cities['NHL'].str.split().str[-3]
    
    nhl_df['team'].replace('\*','',regex=True, inplace=True)
    nhl_df =nhl_df[nhl_df['year']==2018] 
    nhl_df = nhl_df[nhl_df['GP']=='82'] 
    cols = ['GP', 'W', 'L', 'OL', 'PTS', 'PTS%', 'GF', 'GA', 'SRS', 'SOS',
       'RPt%', 'ROW', 'year']
    nhl_df[cols] = nhl_df[cols].apply(pd.to_numeric, errors='coerce')
    nhl_df['W/L%'] = nhl_df['W']/(nhl_df['W']+nhl_df['L'])
    team_pop = nhl_df['team'].str.extract('(\w+$)')
    team_pop.rename(columns = {0 : 'Name team'}, inplace = True)
    matches = team_pop['Name team'].isin(nhl_df['team'].str.split().str[-1])
    team_pop = pd.merge(team_pop, nhl_df[['team','W/L%']], how='inner',left_on='Name team', right_on=nhl_df['team'].str.split().str[-1])
    team_pop = pd.merge(team_pop,cities[['Metropolitan area','Population']], how='left',left_on='Name team',right_on=cities['NHL1'])
    team_pop = pd.merge(team_pop,cities[['Metropolitan area','Population']], how='left',left_on='Name team',right_on=cities['NHL2'])
    team_pop = pd.merge(team_pop,cities[['Metropolitan area','Population']], how='left',left_on='Name team',right_on=cities['NHL3'])
    team_pop= team_pop.fillna('')
    team_pop['Metropolitan area'] = team_pop['Metropolitan area'] +team_pop['Metropolitan area_x'] +team_pop['Metropolitan area_y'] 
    team_pop['Population'] = team_pop['Population'] +team_pop['Population_x'] +team_pop['Population_y']
    team_pop.drop(['Name team', 'Metropolitan area_x','Metropolitan area_y','Population_x','Population_y'], axis=1, inplace=True) 
    team_pop[['W/L%','Population']] = team_pop[['W/L%','Population']].apply(pd.to_numeric, errors='coerce')
    res_df = team_pop.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    population_by_region = res_df['Population'] 
    win_loss_by_region =res_df['W/L%']
    assert len(population_by_region) == len(
        win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region
               ) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nhl_correlation()

0.012486162921209895

# Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NBA using 2018 data.

In [8]:
cities=pd.read_html(r"C:\Users\Юрий\Desktop\Занятия\Coursera\assignments\assignment4\assets\wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.replace('\[.+','',regex=True, inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"([A-Z])", r" \1").str.strip()
cities['MLB'] = cities['MLB'].str.replace(r"([A-Z])", r" \1").str.strip()
cities['NBA'] = cities['NBA'].str.replace(r"([A-Z])", r" \1").str.strip()
cities['NHL'] = cities['NHL'].str.replace(r"([A-Z])", r" \1").str.strip()
cities.rename(columns={"Population (2016 est.)[8]": "Population"},
              inplace=True)
cities['NBA1'] = cities['NBA'].str.split().str[-1]
cities['NBA2'] = cities['NBA'].str.split().str[-2]

nba_df=pd.read_csv(r"C:\Users\Юрий\Desktop\Занятия\Coursera\assignments\assignment4\assets\nba.csv")
nba_df['team'].replace('\*.+|\s\(.+','',regex=True, inplace=True)
nba_df =nba_df[nba_df['year']==2018] 
cols =['W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS', 'year']
nba_df[cols] = nba_df[cols].apply(pd.to_numeric, errors='coerce')
team_pop = nba_df['team'].str.extract('(\w+$)')
team_pop.rename(columns = {0 : 'Name team'}, inplace = True)
#matches = team_pop['Name team'].isin(nba_df['team'].str.split().str[-1])
team_pop = pd.merge(team_pop, nba_df[['team','W/L%']], how='inner',left_on='Name team', right_on=nba_df['team'].str.split().str[-1])
team_pop = pd.merge(team_pop,cities[['Metropolitan area','Population']], how='left',left_on='Name team',right_on=cities['NBA1'])
team_pop = pd.merge(team_pop,cities[['Metropolitan area','Population']], how='left',left_on='Name team',right_on=cities['NBA2'])
team_pop= team_pop.fillna('')
team_pop['Metropolitan area'] =team_pop['Metropolitan area_x'] + team_pop['Metropolitan area_y'] 
team_pop['Population'] =team_pop['Population_x'] + team_pop['Population_y']
team_pop.drop(['Name team', 'Metropolitan area_x','Metropolitan area_y','Population_x','Population_y'], axis=1, inplace=True) 
team_pop[['W/L%','Population']] = team_pop[['W/L%','Population']].apply(pd.to_numeric, errors='coerce')
res_df = team_pop.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
population_by_region = res_df['Population'] 
win_loss_by_region =res_df['W/L%']
stats.pearsonr(population_by_region, win_loss_by_region)[0]

-0.17636350642182938

# Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the MLB using 2018 data.

In [9]:
cities=pd.read_html(r"C:\Users\Юрий\Desktop\Занятия\Coursera\assignments\assignment4\assets\wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.replace('\[.+','',regex=True, inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"([A-Z])", r" \1").str.strip()
cities['MLB'] = cities['MLB'].str.replace(r"([A-Z])", r" \1").str.strip()
cities['NBA'] = cities['NBA'].str.replace(r"([A-Z])", r" \1").str.strip()
cities['NHL'] = cities['NHL'].str.replace(r"([A-Z])", r" \1").str.strip()
cities.rename(columns={"Population (2016 est.)[8]": "Population"},
              inplace=True)
cities['MLB'][3]='Cubs WSox'
cities['MLB'][7]='RSox'
cities['MLB1'] = cities['MLB'].str.split().str[-1]
cities['MLB2'] = cities['MLB'].str.split().str[-2]

mlb_df=pd.read_csv(r"C:\Users\Юрий\Desktop\Занятия\Coursera\assignments\assignment4\assets\mlb.csv")
mlb_df['team'].replace('\*.+|\s\(.+','',regex=True, inplace=True)
mlb_df =mlb_df[mlb_df['year']==2018] 
mlb_df['team'][0] ='Boston RSox'
mlb_df['team'][8] ='Chicago WSox'
cols =['W', 'L', 'W-L%', 'GB', 'year']
mlb_df[cols] = mlb_df[cols].apply(pd.to_numeric, errors='coerce')
team_pop = mlb_df['team'].str.extract('(\w+$)')
team_pop.rename(columns = {0 : 'Name team'}, inplace = True)
team_pop = pd.merge(team_pop, mlb_df[['team','W-L%']], how='inner',left_on='Name team', right_on=mlb_df['team'].str.split().str[-1])
team_pop = team_pop.drop([1, 2], axis='index')
team_pop = pd.merge(team_pop,cities[['Metropolitan area','Population']], how='left',left_on='Name team',right_on=cities['MLB1'])
team_pop = pd.merge(team_pop,cities[['Metropolitan area','Population']], how='left',left_on='Name team',right_on=cities['MLB2'])
team_pop= team_pop.fillna('')
team_pop['Metropolitan area'] =team_pop['Metropolitan area_x'] + team_pop['Metropolitan area_y'] 
team_pop['Population'] =team_pop['Population_x'] + team_pop['Population_y']
team_pop.drop(['Name team', 'Metropolitan area_x','Metropolitan area_y','Population_x','Population_y'], axis=1, inplace=True) 
team_pop[['W-L%','Population']] = team_pop[['W-L%','Population']].apply(pd.to_numeric, errors='coerce')
res_df = team_pop.groupby('Metropolitan area').agg({'W-L%': np.nanmean, 'Population': np.nanmean})
population_by_region = res_df['Population'] 
win_loss_by_region =res_df['W-L%']
stats.pearsonr(population_by_region, win_loss_by_region)[0]

<ipython-input-9-d61fecfe334f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mlb_df['team'][8] ='Chicago WSox'


0.047974279533571064

# Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NFL using 2018 data.

In [30]:
cities=pd.read_html(r"C:\Users\Юрий\Desktop\Занятия\Coursera\assignments\assignment4\assets\wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.replace('\[.+','',regex=True, inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"([A-Z])", r" \1").str.strip()
cities['MLB'] = cities['MLB'].str.replace(r"([A-Z])", r" \1").str.strip()
cities['NBA'] = cities['NBA'].str.replace(r"([A-Z])", r" \1").str.strip()
cities['NHL'] = cities['NHL'].str.replace(r"([A-Z])", r" \1").str.strip()
cities.rename(columns={"Population (2016 est.)[8]": "Population"},
              inplace=True)
cities['NFL1'] = cities['NFL'].str.split().str[-1]
cities['NFL2'] = cities['NFL'].str.split().str[-2]

nfl_df=pd.read_csv(r"C:\Users\Юрий\Desktop\Занятия\Coursera\assignments\assignment4\assets\nfl.csv")
nfl_df =nfl_df[nfl_df['year']==2018]
nfl_df['team'].replace('\*|\+','',regex=True, inplace=True)
nfl_df['team'] = nfl_df['team'].str.strip()
nfl_df.drop([0,5,10,15,20,25,30,35], axis=0,inplace=True )
cols = ['DSRS', 'L', 'MoV', 'OSRS', 'PA', 'PD', 'PF', 'SRS', 'SoS','T', 'W', 'W-L%', 'year']
nfl_df[cols] = nfl_df[cols].apply(pd.to_numeric, errors='coerce')
team_pop = nfl_df['team'].str.extract('(\w+$)')
team_pop.rename(columns = {0 : 'Name team'}, inplace = True)
team_pop = pd.merge(team_pop, nfl_df[['team','W-L%']], how='inner',left_on='Name team', right_on=nfl_df['team'].str.split().str[-1])
team_pop = pd.merge(team_pop,cities[['Metropolitan area','Population']], how='left',left_on='Name team',right_on=cities['NFL1'])
team_pop = pd.merge(team_pop,cities[['Metropolitan area','Population']], how='left',left_on='Name team',right_on=cities['NFL2'])
team_pop= team_pop.fillna('')
team_pop['Metropolitan area'] =team_pop['Metropolitan area_x'] + team_pop['Metropolitan area_y']
team_pop['Population'] =team_pop['Population_x'] + team_pop['Population_y']
team_pop.drop(['Name team', 'Metropolitan area_x','Metropolitan area_y','Population_x','Population_y'], axis=1, inplace=True) 
team_pop[['W-L%','Population']] = team_pop[['W-L%','Population']].apply(pd.to_numeric, errors='coerce')
res_df = team_pop.groupby('Metropolitan area').agg({'W-L%': np.nanmean, 'Population': np.nanmean})
population_by_region = res_df['Population'] 
win_loss_by_region =res_df['W-L%']
stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.00428214143639305

# Question 5
In this question I would like you to explore the hypothesis that given that an area has two sports teams in different sports, those teams will perform the same within their respective sports. How I would like to see this explored is with a series of paired t-tests (so use ttest_rel) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.